In [1]:
from DataManipulation.DataHandler import  mergeMonths, DemaMinDayMultinom

import pandas as pd
from torch.utils.data import ConcatDataset
#data = mergePeriod(1,4,beg="DAT_ASCII_EURUSD_M1_2021",dir="./eurusd2021/",dump=False)
data2021 = mergeMonths(start=1, end=9,beg="DAT_ASCII_EURUSD_M1_2021",dir = './Data/eurusd2021/', dump=False)

dir = "./Data/"
beg = "DAT_ASCII_EURUSD_M1_"

years = range(2010,2021)
dataY = []
for y in years:
    dataY.append(pd.read_csv(dir+beg+str(y)+'.csv',sep=';',names = ['Open','High','Low','Close','Volume']))
dataY.append(data2021)
data = pd.concat(dataY)
data.index.name = "Date"
data.index = pd.to_datetime(data.index)

In [4]:
len(data)

4330453

In [ ]:
from time import time
t = time()
dmc = data.copy()
print("COPY TIME:", time()-t)

emaperiods = [60*2,60*3,60*5,60*7,60*24*100,60*24*200]
datamset = DemaMinDayMultinom(dmc,emaperiods = emaperiods)
print([x.__name__ for x in datamset.indicators])
predictorNames = datamset.periodDiffNames
ResponseName = "_y_enter_exit"

COPY TIME: 0.11122488975524902
9.818594694137573
14.39111876487732
0.7818331718444824
['dema9', 'ema120', 'ema180', 'ema300', 'ema420', 'ema144000', 'ema288000']


In [2]:
dataH = data.resample('1H').agg({'Open': 'first', 
                        'High': 'max', 
                        'Low': 'min', 
                        'Close': 'last'}).dropna()
print(len(dataH),len(data))
from time import time
t = time()
dHc = dataH.copy()
print("COPY TIME:", time()-t)
emaperiods = [100,200,300,24*100,24*200]
dataHset = DemaMinDayMultinom(dHc,emaperiods = emaperiods)

73065 4330453
COPY TIME: 0.006810903549194336
3.620333194732666
0.06895208358764648
0.964195966720581


In [12]:
dataHset.raw['_y_enter_exit'] = dataHset.raw['_y_enter_exit'].to_numpy().astype('int8')
print(dataHset.raw['_y_enter_exit'].to_numpy().dtype)
print(dataHset.raw['_y_enter_exit'].to_numpy().dtype == 'int8')

int8
True


In [20]:
import pandas as pd
import numpy as np
def D1(TS):
    if isinstance(TS,pd.Series):
        D1 = np.zeros(len(TS))
        D1[0] = np.nan
        D1[1:] = TS[1:].to_numpy()-TS[:-1].to_numpy()
        D1 = pd.Series(D1,index = TS.index)
    elif isinstance(TS,np.ndarray):
        D1 = np.zeros(len(TS))
        D1[0] = np.nan
        D1[1:] = TS[1:]-TS[:-1]
    return D1
def D2(TS):
    if isinstance(TS,pd.Series):
        D2 = np.zeros(len(TS))
        npTS = TS.to_numpy()
        D2[:2] = np.nan
        D2[2:] = npTS[2:]-2*npTS[1:-1]+npTS[:-2]
        D2 = pd.Series(D2,index = TS.index)
    elif isinstance(TS,np.ndarray):
        D2 = np.zeros(len(TS))
        D2[:2] = np.nan
        D2[2:] = TS[2:]-2*TS[1:-1]+TS[:-2]
    return D2

In [4]:
dataHset.raw.columns

Index(['Open', 'High', 'Low', 'Close', 'dema9', 'dema9D1', 'dema9D2', 'ema100',
       'ema100D1', 'ema100D2', 'ema200', 'ema200D1', 'ema200D2', 'ema300',
       'ema300D1', 'ema300D2', 'ema2400', 'ema2400D1', 'ema2400D2', 'ema4800',
       'ema4800D1', 'ema4800D2', 'Close_dema9', 'dema9_ema100',
       'dema9D1_ema100D1', 'dema9D2_ema100D2', 'ema100_ema200',
       'ema100D1_ema200D1', 'ema100D2_ema200D2', 'ema200_ema300',
       'ema200D1_ema300D1', 'ema200D2_ema300D2', 'ema300_ema2400',
       'ema300D1_ema2400D1', 'ema300D2_ema2400D2', 'ema2400_ema4800',
       'ema2400D1_ema4800D1', 'ema2400D2_ema4800D2', '_y_enter_exit'],
      dtype='object')

In [21]:
from time import time
import pandas as pd
t = time()
ema100D2_raw = D2(dataHset.raw['ema100'].to_numpy())
ema100D2_ = pd.Series(ema100D2_raw, index = dataHset.raw.index)
print(time()-t)

t = time()
ema100D2_pd = dataHset.raw["ema100D2"]
import matplotlib.pyplot as plt
#plt.plot(ema100D1_raw)
#plt.show()
print(sum(ema100D2_[:]==ema100D2_pd[:]))
print(len(ema100D2_))
print(ema100D2_.isna().sum())
print(ema100D2_.isna().sum()+sum(ema100D2_[:]==ema100D2_pd[:]))
#print(ema100D1_pd[-10:])


0.009894609451293945
72963
73065
102
73065


In [ ]:
raw = dataHset.raw#.dropna()
#print(raw.head())
for i in range(0,20):
    #print(i)
    if (i+1)*24*30*12<len(raw):
        df = raw.iloc[i*24*30*12:((i+1)*24*30*12+(24))]
        plotsubset(df,[x.__name__ for x in dataHset.indicators ],"H"+str(i))
    else:
        df = raw.iloc[i*24*30*12:]
        plotsubset(df,[x.__name__ for x in dataHset.indicators ],"H"+str(i))
        break

In [ ]:
#len(dataset.raw)
ind = np.sign((dataHset.raw["ema100"]-dataHset.raw["ema300"]).to_numpy()) 
stat, count = frameTillMaxMean(
    dataHset.raw['dema9D1'].to_numpy(), 
    ind
)

# 3 class label with threshold on divergence.

In [ ]:
from AnalysisFunc.indanal import zonePriceVarFromStart
from AnalysisFunc.mixinds import split3way
diff3wayind = split3way(
    dataHset.raw['Close'].to_numpy(),
    dataHset.raw['ema2400'].to_numpy(),
    dataHset.raw['ema4800'].to_numpy(),
    0.005
)
stat, count = zonePriceVarFromStart(dataHset.raw['Close'].to_numpy(), diff3wayind)

In [ ]:
import numpy as np
print(len(stat),count)
print(np.mean(stat[stat[:,0]==-1,4:],axis=0))
print(np.mean(stat[stat[:,0]==0,4:] ,axis=0))
print(np.mean(stat[stat[:,0]==1,4:] ,axis=0))

# The above quantiles does indicate a difference in center and average price since the start of a zone. Suggesting to use different strategy in different zones. Plot the whole TS to confirm.

In [ ]:
from AnalysisFunc.plots import plotsubset
raw = dataHset.raw#.dropna()
#print(raw.head())
for i in range(0,20):
    #print(i)
    if (i+1)*24*30*12<len(raw):
        df = raw.iloc[i*24*30*12:((i+1)*24*30*12+(24))]
        diff3wayindsubset =  diff3wayind[i*24*30*12:((i+1)*24*30*12+(24))]
        #print(len(diff3wayindsubset),len(df),np.sum(np.isnan(diff3wayindsubset)),np.sum(np.isnan(diff3wayind)),len((raw)))
        plotsubset(
            df,
            [x.__name__ for x in dataHset.indicators ],
            "H"+str(i), 
            [diff3wayind[i*24*30*12:((i+1)*24*30*12+(24))]]
        )
        
    else:
        df = raw.iloc[i*24*30*12:]
        plotsubset(
            df,
            [x.__name__ for x in dataHset.indicators ],
            "H"+str(i), 
            [diff3wayind[i*24*30*12:]]
        )
        break

# Now after we divided the support into 3 parts, try to see how different strategies gives enter and exit.

First try basic interval base on crossing.

In [ ]:
# given long H+ support, find valid short H+ intervals 
dataHset.raw.columns


In [ ]:
ind = (dataHset.raw['ema100']>dataHset.raw['ema300']) & diff3wayind == 1

In [ ]:
threshold = 0.0005
ind_2 = (dataHset.raw['ema300D1'].to_numpy()>0) | (dataHset.raw['ema100D1'].to_numpy()>0)
ind_1 = (dataHset.raw['ema100']-dataHset.raw['ema300']) > threshold
ind_0 = (diff3wayind == 1)
ind = ind_2 & ind_1 & ind_0

from AnalysisFunc.plots import plotsubset
raw = dataHset.raw#.dropna()
#print(raw.head())
for i in range(0,20):
    #print(i)
    if (i+1)*24*30*12<len(raw):
        df = raw.iloc[i*24*30*12:((i+1)*24*30*12+(24))]
        plotsubset(
            df,
            [x.__name__ for x in dataHset.indicators ],
            "H"+str(i), 
            [
                diff3wayind[i*24*30*12:((i+1)*24*30*12+(24))],
                ind[i*24*30*12:((i+1)*24*30*12+(24))],
                #ind_0[i*24*30*12:((i+1)*24*30*12+(24))],
                #ind_1[i*24*30*12:((i+1)*24*30*12+(24))],
            ]
        )
        
    else:
        df = raw.iloc[i*24*30*12:]
        plotsubset(
            df,
            [x.__name__ for x in dataHset.indicators ],
            "H"+str(i), 
            [
                diff3wayind[i*24*30*12:((i+1)*24*30*12+(24))],
                ind[i*24*30*12:],
                #ind_0[i*24*30*12:],
                #ind_1[i*24*30*12:],
            ]
        )
        break